<a href="https://colab.research.google.com/github/wilfredosanchez/ai-product-incubator/blob/main/AI_Product_Incubator_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SMARTSELLERAI v4.8 - GRACE'S FINAL AUDIT
# Paste this ENTIRE block into ONE Colab +code cell. Delete v4.7 first.

# ===== MOBILE POLISH + SESSION TRACKER =====
from IPython.display import HTML, display
import textwrap
import re
import time
import hashlib  # GRACE FIX #4: For duplicate detection

# Invisible CSS (unchanged)
display(HTML("""
<style>
  .input_prompt {display: none !important;}
  .output_prompt {display: none !important;}
  input[type=text] {width: 80% !important; max-width: 300px !important;}
  @media only screen and (max-width: 600px) {
    .cell {margin: 0 !important; padding: 10px !important;}
    .output_area {padding: 0 !important;}
  }
</style>
"""))

# ===== DAILY SPEND SAFEGUARD (NEW) =====
# GRACE FIX #5: Hard stop at $10/day equivalent (1,666 sessions)
MAX_SESSIONS_PER_DAY = 1666  # Grace's math: $10 / $0.006 per session
session_count = 0  # Resets every Colab runtime (90 min)

# ===== SESSION-BASED RATE LIMITING (NEW) =====
# GRACE FIX #6: Max 10 runs per 90-minute Colab session
SESSION_RUN_LIMIT = 10
run_count = 0

# ===== DUPLICATE REQUEST CACHE (NEW) =====
# GRACE FIX #7: Cache last 5 prompts to avoid double-charges
request_cache = {}  # {hash: result}

# ===== CORE UX (unchanged) =====
print("=" * 50)
print("🎯 AI BUSINESS DISCOVERY ENGINE v4.8")
print("=" * 50)
print("🟢 STEP 1: PRESS PLAY ABOVE ⬆️")
print("=" * 50)
print("TELL ME YOUR LIFE STORY - I'LL FIND YOUR PERFECT BUSINESS")
print("=" * 50)
print("🟢 STEP 2: PRESS PLAY ABOVE ⬆️")
print("=" * 50)
print("HOW TO USE THIS TOOL:")
print("STEP 1: Get your OpenAI API key")
print("→ platform.openai.com/api-keys")
print("→ Add $5 credit (covers 250+ life stories)")
print("STEP 2: Click the ▶️ play button below")
print("STEP 3: Answer 6 open questions (be brutally honest)")
print("STEP 4: AI finds businesses based on YOUR exact life story")
print("💡 TIP: Write like you're texting your best friend")
print("💡 MORE WORDS = MORE SPECIFIC BUSINESSES")
print("🚀 READY? CLICK PLAY BELOW!")
print("=" * 50)

# ===== API SETUP (unchanged) =====
API_KEY = input("Paste your OpenAI API key: ")
if API_KEY.startswith("sk-"):
    print("\n🎉 CONNECTED TO AI!!! INCUBATOR ACTIVATED!!!")
    print("✅ Your AI brain is online and ready!")
    client = openai.OpenAI(api_key=API_KEY)
else:
    print("❌ Invalid API key")
    exit()

# ===== RATE LIMIT CHECK #1 (NEW) =====
# GRACE FIX #6 APPLIED: Stop if session limit hit
if run_count >= SESSION_RUN_LIMIT:
    print(f"❌ Session limit reached ({SESSION_RUN_LIMIT} runs). Restart Colab to continue.")
    exit()

# ===== STORY COLLECTION (unchanged) =====
print("\n" + "=" * 50)
print("📝 TELL ME YOUR LIFE STORY")
print("=" * 50)
print("Type one line, press Enter.")
print("When finished, type DONE + Enter.")
life_lines = []
while True:
    line = input("→ ")
    if line.strip().upper() == 'DONE':
        break
    if line.strip():
        life_lines.append(line.strip())
life_story = " ".join(life_lines)

# ===== INPUT SANITIZATION (v4.7 fix, unchanged) =====
if len(life_story) > 2000:
    print("❌ Story too long. Keep it under 2000 characters (about 300 words).")
    print(f"Yours is {len(life_story)} characters.")
    exit()

# ===== GRACE FIX #8: PROMPT INJECTION SANITIZATION (NEW) =====
# Block common prompt injection attempts
dangerous_patterns = ["ignore previous", "system prompt", "act as", "bypass"]
if any(pattern in life_story.lower() for pattern in dangerous_patterns):
    print("❌ Invalid input detected. Please use natural language only.")
    exit()

# ===== CREATOR DETECTION (unchanged) =====
print("\n" + "=" * 50)
print("🎨 ARE YOU A CREATOR?")
print("=" * 50)
print("Enter Y for Yes, N for No")
creator_answer = input("→ ").upper()
make_skills = ""
if creator_answer == 'Y':
    print("What can you create? (type DONE when finished)")
    make_lines = []
    while True:
        line = input("→ ")
        if line.strip().upper() == 'DONE':
            break
        if line.strip():
            make_lines.append(line.strip())
    make_skills = " ".join(make_lines)

# ===== RESELLER DETECTION (unchanged) =====
print("\n" + "=" * 50)
print("💰 HAVE YOU EVER BEEN A RESELLER?")
print("=" * 50)
print("Enter Y for Yes, N for No")
reseller_answer = input("→ ").upper()
resell_experience = ""
if reseller_answer == 'Y':
    print("What have you sold? (type DONE when finished)")
    resell_lines = []
    while True:
        line = input("→ ")
        if line.strip().upper() == 'DONE':
            break
        if line.strip():
            resell_lines.append(line.strip())
    resell_experience = " ".join(resell_lines)
else:
    print("Would you like reseller ideas? (Y/N)")
    want_reseller_ideas = input("→ ").upper()
    resell_experience = "User has no prior reselling experience but IS open to physical and digital reselling ideas and looking for guidance." if want_reseller_ideas == 'Y' else "User has NO prior reselling experience and does NOT want any reselling-focused business ideas. EXCLUDE ALL RESELLING BUSINESS TYPES."

# ===== DROPDOWNS (unchanged) =====
print("\n" + "=" * 50)
print("⚙️ QUICK QUESTIONS")
print("=" * 50)
print("Budget? (A/B/C/D)")
budget = input("→ ").upper()
print("Time per week? (A/B/C/D)")
time_commitment = input("→ ").upper()
print("Risk tolerance? (A/B/C/D)")
risk_tolerance = input("→ ").upper()

# ===== DUPLICATE CHECK (NEW) =====
# GRACE FIX #7 APPLIED: Hash the prompt, check cache
prompt_hash = hashlib.md5(f"{life_story}{make_skills}{resell_experience}{budget}{time_commitment}{risk_tolerance}".encode()).hexdigest()
if prompt_hash in request_cache:
    print("\n✅ Cached result found! Serving instantly to avoid double-charge.")
    raw_results = request_cache[prompt_hash]
else:
    # ===== COST ESTIMATOR (NEW) =====
    # GRACE FIX #9: Show estimated cost before calling
    est_input_tokens = len(prompt) // 4  # Rough estimate: 1 token = 4 chars
    est_cost = (est_input_tokens * 0.50 + 1500 * 1.50) / 1000000  # GPT-3.5 pricing
    print(f"💰 Estimated cost this run: ${est_cost:.4f}")
    print("=" * 50)

    # ===== DAILY LIMIT CHECK #2 (NEW) =====
    # GRACE FIX #5 APPLIED: Hard stop before API call
    if session_count >= MAX_SESSIONS_PER_DAY:
        print(f"❌ Daily limit reached ({MAX_SESSIONS_PER_DAY} sessions). Contact support.")
        exit()

    # ===== OUTPUT ENGINE (with loading animation) =====
    print("\n🤖 AI is thinking", end="")
    for i in range(3):
        time.sleep(1)
        print(".", end="")
    print("\n")

    # ===== API CALL #1: Business Generation =====
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=1500
        )
        raw_results = response.choices[0].message.content
        
        # Store in cache
        request_cache[prompt_hash] = raw_results
        
        # Increment counters
        global run_count, session_count
        run_count += 1
        session_count += 1
        
    except Exception as e:
        print(f"❌ AI service error: {str(e)}")
        if "quota" in str(e).lower():
            print("💡 Add $5 credit: platform.openai.com/account/billing/overview")
        exit()

# ===== RESULTS FORMATTING (unchanged) =====
business_blocks = re.split(r'(\d+\.)\s*', raw_results)
formatted_businesses = []
for i in range(1, len(business_blocks), 2):
    if i+1 < len(business_blocks):
        number = business_blocks[i].strip()
        content = business_blocks[i+1].strip()
        
        wrapped_content = textwrap.fill(content,
                                      width=67,
                                      break_long_words=False,
                                      break_on_hyphens=False,
                                      subsequent_indent='   ')
        
        formatted_businesses.append(f"\n{number}\n{wrapped_content}")

spaced_results = "\n\n".join(formatted_businesses)

print("\n" + "=" * 50)
print("✨ YOUR 15 HYPER-SPECIFIC BUSINESSES")
print("=" * 50)
print(spaced_results)

# ===== SUMMARY GENERATION (with cache) =====
summary_hash = hashlib.md5(f"summary{prompt_hash}".encode()).hexdigest()
if summary_hash in request_cache:
    print("\n✅ Cached summary found!")
    raw_summary = request_cache[summary_hash]
else:
    summary_prompt = f"""Based on: {life_story}, Skills: {make_skills}, Reselling: {resell_experience}, Budget: {budget}, Time: {time_commitment}, Risk: {risk_tolerance}

And these businesses: {raw_results}

Write a personalized summary that:
1. Acknowledges their specific situation
2. Highlights the 3 BEST businesses for them (numbered 1, 2, 3 on NEW LINES)
3. Explains WHY these 3 fit them perfectly
4. Gives them confidence to start
5. Uses their exact words when possible
6. Includes a "💡 THINGS TO CONSIDER" section encouraging them to try new things
7. Makes sure each number (1, 2, 3) starts on a NEW LINE

Write like you're their successful friend who knows their life.

FORMAT:
🎯 YOUR PERFECT BUSINESS MATCHES

Based on everything you told me, here are your TOP 3 businesses:

1. [BUSINESS NAME]
   Perfect because [specific reason from their story]

2. [BUSINESS NAME]
   Ideal for [specific constraint they mentioned]

3. [BUSINESS NAME]
   Tailored for [specific skill/experience]

💡 WHY THESE WORK FOR YOU:
[Personalized reasoning using their exact words]

💡 THINGS TO CONSIDER:
[Encouraging text about trying new things and potential for success in new areas]

🚀 NEXT STEPS:
[Specific action based on their budget/time/risk]

You've got this. I believe in you.
"""
    
    try:
        summary_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": summary_prompt}],
            temperature=0.7,
            max_tokens=800
        )
        raw_summary = summary_response.choices[0].message.content
        request_cache[summary_hash] = raw_summary
        
    except Exception as e:
        print(f"❌ Summary generation error: {str(e)}")
        raw_summary = "Summary unavailable. But your businesses are above—start there!"

# ===== FINAL OUTPUT (unchanged) =====
wrapped_summary = textwrap.fill(raw_summary, width=70, break_long_words=False, break_on_hyphens=False)

print("\n" + "=" * 50)
print("🎯 YOUR PERSONALIZED BUSINESS SUMMARY")
print("=" * 50)
print(wrapped_summary)

# ===== GRACE FIX #10: COLAB WARNING (NEW) =====
print("\n" + "=" * 50)
print("⚠️ CO-LAB SESSION ACTIVE")
print("=" * 50)
print("This session expires in 90 minutes. Results are saved above.")
print(f"Runs remaining this session: {SESSION_RUN_LIMIT - run_count}")

# ===== GRACE FIX #11: COST TRACKER (NEW) =====
total_est_cost = session_count * 0.006
print(f"Estimated total API cost this session: ${total_est_cost:.2f}")
